In [1]:
from google.colab import drive
drive.mount("/content/drive")
# %cd /content/drive/Shareddrives/Duong-LongWarwick/ROM
# !python -m atari_py.import_roms .
%cd '/content/drive/Shareddrives/Duong-KhoiBKHN/PantheonRL-Environment'
!ls

Mounted at /content/drive
/content/drive/Shareddrives/Duong-KhoiBKHN/PantheonRL-Environment
OvercookedAI-Environment.ipynb	PantheonRL-MockEnvironment.ipynb
overcooked_ai_py


In [3]:
# from environment import OvercookedEnvironment
# from gym_cooking.envs import OvercookedEnvironment
# from recipe_planner.recipe import *

import numpy as np
import random
import argparse
from collections import namedtuple

import gym


In [4]:
class manycook_arg:
    def __init__(self):
        # Environment
        self.level = 'open-divider_salad' 
        self.num_agents = 2 
        self.max_num_timesteps = 100
        self.max_num_subtasks = 14
        self.seed = 1
        self.with_image_obs = False
        self.beta = 1.3

        # Navigation Planner
        self.alpha = 0.01
        self.tau = 2
        self.cap = 75
        self.main_cap = 100

        # Visualizations 
        self.play = True
        self.record = False
        
        # Models 
        self.model1 = None 
        self.model2 = None 
        self.model3 = None 
        self.model4 = None